# Predict Future Sales

This script contains the code for modelling sales data per month. Stationarity and Seasonality are explored, and an ARIMA model is set up, as well as a Prophet model. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
data=pd.read_csv("sales_train_v2.csv", parse_dates=['date'])
ts=data.groupby(["date"])["item_cnt_day"].sum()
ts=ts.reset_index()
ts.head()
data.dtypes
ts.head()

Note that this data sums total company sales per month. Sales per day can also be explored by changed the code to:


In [ ]:
#ts=data.groupby(["date_block_num"])["item_cnt_day"].sum()
#ts.index=pd.date_range(start = '2013-01-01',end='2015-10-01', freq = 'MS')
#ts=ts.reset_index()

In [ ]:
data=pd.read_csv("sales_train_v2.csv", parse_dates=['date'])

This function tests the stationarity (ensuring mean, variance, autocorrelation) of the time series data. The data will be plotted, along with the rolling mean and rolling standard deviation of the data(the window can be adjusted accordingly in the function). The Dickey fuller test can be used as a further test of stationarity. The 'Test Statistic' should be negative, and less than the 5% critical value provided, the p-value can also be checked, and should be less than 0.05.

In [ ]:
def test_stationarity(timeseries):
    
    #Determing rolling statistics
    rolmean = pd.rolling_mean(timeseries, window = 12)
    rolstd = pd.rolling_std(timeseries, window = 12)

    #Plot rolling statistics:
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

    

In [ ]:
test_stationarity(ts)

Investigate the log transformation of the data, and see if this makes the data look more stationary

In [ ]:
log_ts=log(ts)
test_stationarity(log_ts)

Try removing moving average from the data, to test stationarity, again the window can be changed depending on the period of the time series. NA values will be created for the first 12 values, so these need dropped to investigate the stationarity

In [ ]:
moving_avg = pd.rolling_mean(ts,12)
plt.plot(ts)
plt.plot(moving_avg, color='red')

ts_moving_avg_diff = ts - moving_avg
ts_moving_avg_diff.head(12)

ts_moving_avg_diff.dropna(inplace=True)
test_stationarity(ts_moving_avg_diff)

Try shifting the data one place(can also be shifted more places), again 1 NA value will be created and will need to be omitted

In [ ]:
ts_diff = ts - ts.shift()
plt.plot(ts_diff)

In [ ]:
ts_diff.dropna(inplace=True)
test_stationarity(ts_diff)

# Exploring Seasonality

Next, seasonality needs to be explored. Seasonality is the occurance of variation in time series data at a recurring and predictable time within a period, often a year. Since this data is monthly, the freq=12 for 12 months in a year

In [ ]:
decomposition = seasonal_decompose(ts, freq=12)
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

Plotting the trend, seasonality and residual components of the time series against the original graph

In [ ]:
plt.subplot(411)
plt.plot(ts, label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residuals')
plt.legend(loc='best')
plt.tight_layout()

# Autocorrelation and Partial Autocorrelation Plots

At lag k, this is the correlation between series values that are k intervals apart.

Partial autocorrelation function (PACF). At lag k, this is the correlation between series values that are k intervals apart, accounting for the values of the intervals between.

In [ ]:
lag_acf = acf(ts_diff, nlags=20)
lag_pacf = pacf(ts_diff, nlags=20, method='ols')

#Plot ACF
plt.subplot(121) 
plt.plot(lag_acf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts_diff)),linestyle='--',color='gray')
plt.title('Autocorrelation Function')

#Plot PACF:
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts_diff)),linestyle='--',color='gray')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()

# ARIMA Modelling

This is the code for the ARIMA model. This is the area I am struggling with, as the predictions are not good. The parameters of the model can be changed (order=(p,d,q)) where p – number of time lags, d – degree of differencing and q – order of moving average model.

The plots plot the data and the results of the ARIMA model

In [ ]:
model = ARIMA(ts, order=(1, 1, 1))  
results_ARIMA = model.fit(disp=-1)  
plt.plot(ts_diff)
plt.plot(results_ARIMA.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_ARIMA.fittedvalues-ts_diff)**2))

This adjusts the data if it was lagged

In [ ]:
predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues, copy=True)
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
print( predictions_ARIMA_diff_cumsum.head())

This re-indexes the results of the model

In [ ]:
predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues, copy=True)
print (predictions_ARIMA_diff.head())
predictions_ARIMA = pd.Series(ts.ix[0], index=ts.index)
predictions_ARIMA = predictions_ARIMA.add(predictions_ARIMA_diff_cumsum,fill_value=0)
predictions_ARIMA.head()

Take the exponent if the TS was given a log transformation

In [ ]:
predictions_ARIMA = np.exp(predictions_ARIMA_log)
plt.plot(ts)
plt.plot(predictions_ARIMA)
plt.title('RMSE: %.4f'% np.sqrt(sum((predictions_ARIMA-ts)**2)/len(ts)))


# Prophet Model (Total Company Sales)

The Prophet model for total company sales per month

The RMSE needs to be found for these predictions

In [ ]:
from fbprophet import Prophet
#Creating Appropriate Dataframe #
proph = data.groupby(['date_block_num'])[ 'item_cnt_day'].sum()
proph.index=pd.date_range(start='2013-01-01', end='2015-10-01', freq='MS')
proph = proph.to_frame().reset_index()
proph.columns = ['ds', 'y']
proph.head()
#Modelling#
model=Prophet(yearly_seasonality=True)
model.fit(proph)
#Making future predictions
future_data = model.make_future_dataframe(periods=1, freq='MS')
forecast_data = model.predict(future_data)
forecast = model.predict(future_data)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
#Plotting Results
model.plot(forecast_data)
model.plot_components(forecast_data)

# Prophet Model (Individual Store Sales)

This Prophet Model predicts the total sales of each store in the data set.

The RMSE needs to be found for these predictions


In [ ]:
monthly_shop_sales=data.groupby(["date_block_num","shop_id"])["item_cnt_day"].sum()
# get the shops to the columns
monthly_shop_sales=monthly_shop_sales.unstack(level=1)
monthly_shop_sales=monthly_shop_sales.fillna(0)
monthly_shop_sales.index=dates
monthly_shop_sales=monthly_shop_sales.reset_index()
monthly_shop_sales.head()

forecastsDict = {}
for node in range(len(monthly_shop_sales)):
    # take the date-column and the col to be forecasted
    nodeToForecast = pd.concat([monthly_shop_sales.iloc[:,0], monthly_shop_sales.iloc[:, node+1]], axis = 1)
    # rename for prophet compatability
    nodeToForecast = nodeToForecast.rename(columns = {nodeToForecast.columns[0] : 'ds'})
    nodeToForecast = nodeToForecast.rename(columns = {nodeToForecast.columns[1] : 'y'})
    growth = 'linear'
    m = Prophet(growth, yearly_seasonality=True)
    m.fit(nodeToForecast)
    future = m.make_future_dataframe(periods = 1, freq = 'MS')
    forecastsDict[node] = m.predict(future)

nCols = len(list(forecastsDict.keys()))+1
for key in range(0, nCols-1):
    f1 = np.array(forecastsDict[key].yhat)
    f2 = f1[:, np.newaxis]
    if key==0:
        predictions=f2.copy()
       # print(predictions.shape)
    else:
       predictions = np.concatenate((predictions, f2), axis = 1)
       
predictions_unknown=predictions[-1]
predictions_unknown

# XGBoost Model (Individual Item-Store Predictions)

This XGBoost model could predict the next months sales of each item in each store. This code is running errors I can't work out, therefore I havent been able to tune the parameters to create the best predictions. I think it just needs something tweaking to get it to work

In [ ]:
import xgboost as xgb
param = {'max_depth':10, 
         'subsample':1,
         'min_child_weight':0.5,
         'eta':0.3, 
         'num_round':1000, 
         'seed':1,
         'silent':0,
         'eval_metric':'rmse'}

In [ ]:
xgbtrain = xgb.DMatrix(train.iloc[:,  (train.columns != 33)].values, train.iloc[:, train.columns == 33].values)
watchlist  = [(xgbtrain,'train-rmse')]

bst = xgb.train(param, xgbtrain)
preds = bst.predict(xgb.DMatrix(train.iloc[:,  (train.columns != 33)].values))
from sklearn.metrics import mean_squared_error 
rmse = np.sqrt(mean_squared_error(preds,train.iloc[:, train.columns == 33].values))
print(rmse)
xgb.plot_importance(bst)
apply_df = test
apply_df['shop_id']= apply_df.shop_id.astype('str')
apply_df['item_id']= apply_df.item_id.astype('str')

apply_df = test.merge(train_cleaned_df, how = "left", on = ["shop_id", "item_id"]).fillna(0.0)
d = dict(zip(apply_df.columns[4:],list(np.array(list(apply_df.columns[4:])) - 1)))